In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import shapiro, levene, f_oneway, kruskal, mannwhitneyu, ttest_ind
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('dataset_final_cleaned.csv')
df.columns = df.columns.str.strip().str.lower()

In [106]:
df["released_dt"] = pd.to_datetime(df["released"], errors="coerce")
df["updated_dt"] = pd.to_datetime(df["updated"], errors="coerce")
df["release_month_clean"] = df["released_dt"].dt.month
snapshot_date = df["updated_dt"].max()
df["age_days"] = (snapshot_date - df["released_dt"]).dt.days
df = df[df["release_month_clean"].between(1, 12)].copy()
df = df[df["age_days"].notna() & (df["age_days"] > 0)].copy()

# Нормализованные метрики
df["ratings_per_day"] = df["ratings_count"] / df["age_days"]
df["suggestions_per_day"] = df["suggestions_count"] / df["age_days"]
df["added_per_day"] = df["added"] / df["age_days"]


# Блок 1: Первоначальная гипотеза

## Гипотеза: Апрель наиболее выгодный для релиза нежели ноябрь

$H_0$: Апрель и ноябрь одниковые с точки зрения популярности игр.

$H_1$: В апреле популярность игр выше, чем в ноябре.

Мы рассматриваем именно апрель и ноябрь, потому что апрель по результатам EDA является фаваритом с хорошей стороны, а ноябрь - с плохой стороны. Значит, разница будет максимальной, и мы точно увидим статистический эффект.

Мы сравним месяцы по трем признакам популярности: оценок в день, рекомендаций в день и добавлений в день.

## Сравнение по оценкам в день

In [111]:
group_a = df[df['release_month_clean'] == 11]['ratings_per_day']
group_b = df[df['release_month_clean'] == 4]['ratings_per_day']

In [112]:
q_low_a, q_high_a = group_a.quantile(0.05), group_a.quantile(0.95)
q_low_b, q_high_b = group_b.quantile(0.05), group_b.quantile(0.95)

group_a_clean = group_a[(group_a >= q_low_a) & (group_a <= q_high_a)]
group_b_clean = group_b[(group_b >= q_low_b) & (group_b <= q_high_b)]

In [113]:
print(f"Группа A (Ноябрь): {len(group_a_clean)} игр")
print(f"Группа B (Апрель): {len(group_b_clean)} игр")

Группа A (Ноябрь): 266 игр
Группа B (Апрель): 184 игр


Выборки имеют нормальный размер, а также они сбалансированы. Теперь проверим на нормальность с помощью теста Шапиро-Уилка.

In [114]:
stat_a, p_a = stats.shapiro(group_a_clean)
print(f"Группа A: statistic={stat_a:.4f}, p-value={p_a:.4f}")
stat_b, p_b = stats.shapiro(group_b_clean)
print(f"Группа B: statistic={stat_b:.4f}, p-value={p_b:.4f}")

Группа A: statistic=0.3475, p-value=0.0000
Группа B: statistic=0.6468, p-value=0.0000


Мы видим, что p-value для обоих групп меньше 0.05, что говорит о том, что распределение не нормальное. После этого определим коэффциент вариации.

In [115]:
cv_a = (group_a_clean.std() / group_a_clean.mean()) * 100
cv_b = (group_b_clean.std() / group_b_clean.mean()) * 100

print(f"Группа A (Ноябрь): CV = {cv_a:.2f}%")
print(f"Группа B (Апрель): CV = {cv_b:.2f}%")

   Группа A (Ноябрь): CV = 309.27%
   Группа B (Апрель): CV = 144.96%


Мы видим, что в обоих группах вариативность очень большая. Это говорит о том, что наши результаты могут быть некой случайнностью. Для того, чтобы более корректно оценить практический эффект необходимо сранивать медианы, а не среднее выборок.

В завершение проверим равенство дисперсии по тесту Левена.

In [116]:
stat_var, p_var = stats.levene(group_a_clean, group_b_clean)
print(f"Statistic: {stat_var:.4f}, p-value: {p_var:.4f}")

   Statistic: 8.5461, p-value: 0.0036


Мы видим, что значение p-value < 0.05, следовательно, дисперсии не равны.

Поскольку у нас не нормальное распределение, то применим непараметрический статистический тест Манна-Уитни к нашим выборкам.

In [ ]:
u_stat, p_value = stats.mannwhitneyu(group_b_clean, group_a_clean, alternative='greater')
print(f"Statistic: {u_stat:.4f}")
print(f"p-value: {p_value:.4f}")

   Statistic: 30284.0000
   p-value: 0.0000


Покольку p-value примерно ноль, то мы понимаем, нулевая гипотеза отвергается. Это доказывает, что присутствует заметная сезонность.Определим ее практический эффект при помощи медиан выборок.

In [ ]:
print("   Базовые статистики:")
print(f"   Ноябрь (A): медиана = {group_a_clean.median():.4f}")
print(f"   Апрель (B): медиана = {group_b_clean.median():.4f}")


   Базовые статистики:
   Ноябрь (A): медиана = 0.0057
   Апрель (B): медиана = 0.0108


Мы видим, что медиана почти в два раза больше у апреля. **Это подтверждает, что у данных есть заметная сезонность и наиболее благоприятным для релиза является апрель.**

## Рекомендаций в день

In [117]:
group_a = df[df['release_month_clean'] == 11]['suggestions_per_day']
group_b = df[df['release_month_clean'] == 4]['suggestions_per_day']

In [118]:
q_low_a, q_high_a = group_a.quantile(0.05), group_a.quantile(0.95)
q_low_b, q_high_b = group_b.quantile(0.05), group_b.quantile(0.95)

group_a_clean = group_a[(group_a >= q_low_a) & (group_a <= q_high_a)]
group_b_clean = group_b[(group_b >= q_low_b) & (group_b <= q_high_b)]

In [119]:
print(f"Группа A (Ноябрь): {len(group_a_clean)} игр")
print(f"Группа B (Апрель): {len(group_b_clean)} игр")

Группа A (Ноябрь): 264 игр
Группа B (Апрель): 184 игр


Выборки имеют нормальный размер, а также они сбалансированы. Теперь проверим на нормальность с помощью теста Шапиро-Уилка.

In [120]:
stat_a, p_a = stats.shapiro(group_a_clean)
print(f"Группа A: statistic={stat_a:.4f}, p-value={p_a:.4f}")
stat_b, p_b = stats.shapiro(group_b_clean)
print(f"Группа B: statistic={stat_b:.4f}, p-value={p_b:.4f}")

Группа A: statistic=0.5122, p-value=0.0000
Группа B: statistic=0.9311, p-value=0.0000


Мы видим, что p-value для обоих групп меньше 0.05, что говорит о том, что распределение не нормальное. После этого определим коэффциент вариации.

In [122]:
cv_a = (group_a_clean.std() / group_a_clean.mean()) * 100
cv_b = (group_b_clean.std() / group_b_clean.mean()) * 100

print(f"Группа A (Ноябрь): CV = {cv_a:.2f}%")
print(f"Группа B (Апрель): CV = {cv_b:.2f}%")


   Группа A (Ноябрь): CV = 220.13%
   Группа B (Апрель): CV = 54.47%


Мы видим, что в обоих группах вариативность большая. Это говорит о том, что наши результаты могут быть некой случайнностью. Для того, чтобы более корректно оценить практический эффект необходимо сранивать медианы, а не среднее выборок.

В завершение проверим равенство дисперсии по тесту Левена.

In [123]:
stat_var, p_var = stats.levene(group_a_clean, group_b_clean)
print(f"Statistic: {stat_var:.4f}, p-value: {p_var:.4f}")

Statistic: 37.1793, p-value: 0.0000


Мы видим, что значение p-value < 0.05, следовательно, дисперсии не равны.

Поскольку у нас не нормальное распределение, то применим непараметрический статистический тест Манна-Уитни к нашим выборкам.

In [ ]:
u_stat, p_value = stats.mannwhitneyu(group_b_clean, group_a_clean, alternative='greater')
print(f"Statistic: {u_stat:.4f}")
print(f"p-value: {p_value:.4f}")

   Statistic: 25295.0000
   p-value: 0.2276


Мы видим, p-value > 0.05. Значит, наши выборки статистически одинаковые, то есть подверждается нулеая гипотеза.

Это говорит о том, что **для рекомендация в день все месяцы одниковые.**

## Добавлений в день

In [124]:
group_a = df[df['release_month_clean'] == 11]['added_per_day']
group_b = df[df['release_month_clean'] == 4]['added_per_day']

In [125]:
q_low_a, q_high_a = group_a.quantile(0.05), group_a.quantile(0.95)
q_low_b, q_high_b = group_b.quantile(0.05), group_b.quantile(0.95)

group_a_clean = group_a[(group_a >= q_low_a) & (group_a <= q_high_a)]
group_b_clean = group_b[(group_b >= q_low_b) & (group_b <= q_high_b)]

In [126]:
print(f"Группа A (Ноябрь): {len(group_a_clean)} игр")
print(f"Группа B (Апрель): {len(group_b_clean)} игр")

Группа A (Ноябрь): 264 игр
Группа B (Апрель): 184 игр


Выборки имеют нормальный размер, а также они сбалансированы. Теперь проверим на нормальность с помощью теста Шапиро-Уилка.

In [127]:
stat_a, p_a = stats.shapiro(group_a_clean)
print(f"Группа A: statistic={stat_a:.4f}, p-value={p_a:.4f}")

stat_b, p_b = stats.shapiro(group_b_clean)
print(f"Группа B: statistic={stat_b:.4f}, p-value={p_b:.4f}")

   Группа A: statistic=0.2723, p-value=0.0000
   Группа B: statistic=0.7627, p-value=0.0000


Мы видим, что p-value для обоих групп меньше 0.05, что говорит о том, что распределение не нормальное. После этого определим коэффциент вариации.

In [128]:
cv_a = (group_a_clean.std() / group_a_clean.mean()) * 100
cv_b = (group_b_clean.std() / group_b_clean.mean()) * 100

print(f"Группа A (Ноябрь): CV = {cv_a:.2f}%")
print(f"Группа B (Апрель): CV = {cv_b:.2f}%")


Группа A (Ноябрь): CV = 367.56%
Группа B (Апрель): CV = 105.34%


Мы видим, что в обоих группах вариативность большая. Это говорит о том, что наши результаты могут быть некой случайнностью. Для того, чтобы более корректно оценить практический эффект необходимо сранивать медианы, а не среднее выборок.

В завершение проверим равенство дисперсии по тесту Левена.

In [129]:
stat_var, p_var = stats.levene(group_a_clean, group_b_clean)
print(f"Statistic: {stat_var:.4f}, p-value: {p_var:.4f}")

Statistic: 7.3909, p-value: 0.0068


Мы видим, что значение p-value < 0.05, следовательно, дисперсии не равны.

Поскольку у нас не нормальное распределение, то применим непараметрический статистический тест Манна-Уитни к нашим выборкам.

In [131]:
u_stat, p_value = stats.mannwhitneyu(group_b_clean, group_a_clean, alternative='greater')
print(f"Statistic: {u_stat:.4f}")
print(f"p-value: {p_value:.4f}")

Statistic: 28980.0000
p-value: 0.0002


Покольку p-value примерно ноль, то мы понимаем, нулевая гипотеза отвергается. Это доказывает, что присутствует заметная сезонность. Определим ее практический эффект при помощи медиан выборок.

In [132]:
print(f"Ноябрь (A): медиана = {group_a_clean.median():.4f}")
print(f"Апрель (B): медиана = {group_b_clean.median():.4f}")


Ноябрь (A): медиана = 0.1028
Апрель (B): медиана = 0.1382


Мы видим, что медиана апреля заметно больше медианы ноября. Это говорит, что добавлений в день есть сезонность.

**Подводя итог по всей гипотезы, мы можем сказать, что по двум из трех параметров наблюдается сезонность, пик которой приходится на апрель.**

# Гипотеза: Июль (низкая конкуренция) vs октябрь (высокая конкуренция)

$H_0$: Уровень конкуренции не влияет на эффективность релиза.

$H_1$: В июле эффективность выше, чем в октябре.

Для анализа мы выбрали июль, так как в нем самая маленькое кол-во релизов, а в октябре второе по значению. Первое у ноября, который в целом довольно странный, поэтому для достоверности анализа возьмем октябрь.

Эффективность мы будем рассчитывать по формуле $Value~ Score = \frac{ratings~per~day}{initial~price}$.


Эта формула показывает, сколько оценки приносит 1 доллар. С точки зрения бизнеса это говорит о том, что каждая оценка показывает игрокам, что "я доиграл до момента, когда готов оценить", то есть игрокам интересно в нее играть и они проиграли достаточно много, чтобы смочь оценить игру. Делим на цену, так как "крутость" игры зависит в том числе и от цены.



Для анализа мы будем использовать только те игры, которые выпускают не мажорные разработчки, поскольку у инди разработчиков нет возможности "купить" рекламу своей игре, и поэтому для них наиболее важно в какой момент выпускать игру. Значит, на их примере результат будет наиболее честный для компаний, которые соревнуться не в бюджете на маркетинг, а в качестве игры.

In [137]:
df_indie = df[df['major_publisher'] == False]

df_indie['value_score'] = df_indie['ratings_per_day'] / df_indie['initial_price'].clip(lower=1)


group_a = df_indie[df_indie['release_month_clean'] == 10]['value_score']
group_b = df_indie[df_indie['release_month_clean'] == 7]['value_score']

group_a_clean = group_a[group_a.between(group_a.quantile(0.05), group_a.quantile(0.95))]
group_b_clean = group_b[group_b.between(group_b.quantile(0.05), group_b.quantile(0.95))]

print(f"Группа A (Октябрь): {len(group_a_clean)} игр")
print(f"Группа B (Июль): {len(group_b_clean)} игр")

Группа A (Октябрь): 235 игр
Группа B (Июль): 156 игр


Мы видим что данных в группах достаточно для стастического анализа. Также они достаточно сбаланнсированы, хоть может и быть более приятная картина. Определим относительную конкуренцию между месяцами.

In [139]:
comp_sep = len(df_indie[df_indie['release_month_clean'] == 10])
comp_jul = len(df_indie[df_indie['release_month_clean'] == 7])
print(f"Конкуренция: Октябрь={comp_sep}, Июль={comp_jul}")
print(f"Конкуренция выше в {comp_sep/comp_jul:.1f} раза в октябре")
print()

Конкуренция: Октябрь=248, Июль=174
Конкуренция выше в 1.4 раза в октябре



Мы видим, что конкуренция достаточно значимая, что показывает ценность и актуальность данной проверки.

Для дальнейшей работы проверим, нормальны ли наши выборки при помощи теста Шапиро-Уилка?

In [140]:
stat_a, p_a = stats.shapiro(group_a_clean)
print(f"Октябрь: p-value={p_a:.4f}")

stat_b, p_b = stats.shapiro(group_b_clean)
print(f"Июль: p-value={p_b:.4f}")

Октябрь: p-value=0.0000
Июль: p-value=0.0000


Мы видим, что p-value для обоих групп меньше 0.05, что говорит о том, что распределение не нормальное. После этого определим коэффциент вариации.

In [142]:
cv_a = (group_a_clean.std() / group_a_clean.mean()) * 100
cv_b = (group_b_clean.std() / group_b_clean.mean()) * 100

print(f"Октябрь: CV = {cv_a:.2f}%")
print(f"Июль: CV = {cv_b:.2f}%")
print()

Октябрь: CV = 130.12%
Июль: CV = 119.88%



Мы видим, что в обоих группах вариативность большая. Это говорит о том, что наши результаты могут быть некой случайнностью. Для того, чтобы более корректно оценить практический эффект необходимо сранивать медианы, а не среднее выборок.

В завершение проверим равенство дисперсии по тесту Левена.

In [145]:
stat_var, p_var = stats.levene(group_a_clean, group_b_clean)
print(f"p-value={p_var:.4f}")

p-value=0.7726


Мы видим, что дисперсии равны, так как p-value > 0.05. Однако, наши распределения все равно являются не нормально распределенными, мы будем использовать тест Манна-Уитни для проверки гипотезы.

In [146]:
u_stat, p_value = stats.mannwhitneyu(group_b_clean, group_a_clean, alternative='greater')
print(f"p-value: {p_value:.4f}")

p-value: 0.0308


Покольку p-value < 0.05, то мы понимаем, нулевая гипотеза отвергается. Это доказывает, что конкуренция влияет на эффективность. Определим ее практический эффект при помощи медиан выборок.

In [148]:
print(f"Октяборь: медиана = {group_a_clean.median():.4f}")
print(f"Июль: медиана = {group_b_clean.median():.4f}")

Октяборь: медиана = 0.0006
Июль: медиана = 0.0008


Мы видим, что практический эффект очень минимальный, то есть на практике наши группы имеют одниковое значение, и практически не различимы. Это говорит нам о том, что на практике нулевая гипотеза верна и нет различий в эффективности в разные периоды конкуренции.

**Вывод: Стратегия вывода игры на рынок зависит не только от месяца, но еще и от жанра и целей компании.**

# Блок 2: Анализ жанровой специфики

## Гипотеза: Жанр Active: в марте выше, чем качество игры в октябре

$H_0$: Качество игры в марте и декабре одинаково.

$H_1$: В марте игры качественнее, чем в оятбре.

Из EDA анализа мы видим, что наиболее заметный жанр - это Active, и внутри этого жанра интересно проверить, есть ли месяцы, в которые выходят более качественные игры. По анализу хочется выбрать ноябрь, вместо откбря, однако в ноябре в три раза больше данных, поэтому группы будут не сбалансированы. Следовательно, тестирование провести не получится. Декабрь нам не подходит, поскольку там очень мало данных, и они не могут давать хорошей гарантии и статистики. Откябрь является наиболее близким к ноябрю, поэтому выбираем его для дальнейшего анализа.



Для анализа мы будем рассматривать платные игры, которые имеют небольшое кол-во платформу, потому что:

* к платным игрым другое отношение в отличае от бесплатных (более осознаное оценивание)

* на большом количестве платформ выспускаются с игры с огромным бюджетом. Такие игры не отображают реального положения на рынке.

In [151]:
df_action = df[df['main_genre'] == 'Action'].copy()

df_filtered = df_action[
    (df_action['ratings_count'] >= 10) &
    (df_action['platform_count'] <= 3)
]

group_a = df_filtered[df_filtered['release_month_clean'] == 10]['weighted_rating']
group_b = df_filtered[df_filtered['release_month_clean'] == 3]['weighted_rating']

print(f"Всего Action игр в датасете: {len(df_action)}")
print(f"После фильтрации: {len(df_filtered)}")
print()

print(f"Группа A (Октябрь): {len(group_a)} игр")
print(f"Группа B (Март): {len(group_b)} игр")



Всего Action игр в датасете: 323
После фильтрации: 122

Группа A (Октябрь): 9 игр
Группа B (Март): 8 игр


Получили очень маленькое кол-во данных, поэтому статистический тест некорректно применять, поскольку значения могут абсолютно случайными.

**Вывод: для данной гипотезы невозможно провести A/B-тест.**


# Блок 4: Платформенные гипотезы

В данном блоке все гипотезы будут направлены на то, чтобы проанализировать сезонность наиболее популярных платформ.

В качестве наиболее "качественного" месяца мы будем брать тот, что имеет самую большую медиану взвешенного рейтинга на тепловой карте "Месяц х Плтаформа", а в качестве наиболее "некачественного" месяц, где самая маленькая медиана.

## Android: качество игр в январе vs качество игр в мае

$H_0$: Качество игр в январе и мае одинаковое.

$H_1$: Игры в январе качественне, чем в мае.

In [152]:
df['is_android'] = df['platforms'].str.contains('Android', case=False, na=False)
df_android = df[df['is_android'] == True].copy()

df_filtered = df_android[
    (df_android['ratings_count'] >= 10) &
    (df_android['initial_price'] > 0) &
    (df_android['platform_count'] <= 3)
]

print(f"Всего Android игр: {len(df_android)}")
print(f"После фильтрации: {len(df_filtered)}")
print()

group_a = df_filtered[df_filtered['release_month_clean'] == 5]['weighted_rating']
group_b = df_filtered[df_filtered['release_month_clean'] == 1]['weighted_rating']

print(f"Группа A (Май): {len(group_a)} игр")
print(f"Группа B (Январь): {len(group_b)} игр")

Всего Android игр: 299
После фильтрации: 27

Группа A (Май): 2 игр
Группа B (Январь): 4 игр


Игр маленькое количество, поэтому дальнейший анализ невозможен из-за маленькой выборки, поскольку маленькая выборка приводит к случайности результата.

**Вывод: для данной гипотезы невозможно провести A/B-тест.**


## Xbox One: качество игр в марте vs качество игр в сентябре

$H_0$: Качество игр в марте и сентябре одинаковое.

$H_1$: Игры в марте качественне, чем в сентябре.

In [153]:
df['is_xbox_one'] = df['platforms'].str.contains('Xbox One', case=False, na=False)
df_xbox = df[df['is_xbox_one'] == True].copy()

df_filtered = df_xbox[
    (df_xbox['ratings_count'] >= 10) &
    (df_xbox['initial_price'] > 0) &
    (df_xbox['platform_count'] <= 4)
]

print(f"Всего Xbox One игр: {len(df_xbox)}")
print(f"После фильтрации: {len(df_filtered)}")
print()

group_a = df_filtered[df_filtered['release_month_clean'] == 9]['weighted_rating']
group_b = df_filtered[df_filtered['release_month_clean'] == 3]['weighted_rating']

print(f"Группа A (Сентябрь): {len(group_a)} игр")
print(f"Группа B (Март): {len(group_b)} игр")

Всего Xbox One игр: 594
После фильтрации: 193

Группа A (Сентябрь): 16 игр
Группа B (Март): 20 игр


Игр маленькое количество, поэтому дальнейший анализ невозможен из-за маленькой выборки, поскольку маленькая выборка приводит к случайности результата.

**Вывод: для данной гипотезы невозможно провести A/B-тест.**


## PlayStation 4: качество игр в марте vs качество игр в сентябре

$H_0$: Качество игр в марте и сентябре одинаковое.

$H_1$: Игры в марте качественне, чем в сентябре.

In [154]:
df['is_ps4'] = df['platforms'].str.contains('PlayStation 4', case=False, na=False)
df_ps4 = df[df['is_ps4'] == True].copy()

df_filtered = df_ps4[
    (df_ps4['ratings_count'] >= 10) &
    (df_ps4['initial_price'] > 0) &
    (~df_ps4['is_free']) &
    (df_ps4['platform_count'] <= 4)
]

print(f"Всего PS4 игр: {len(df_ps4)}")
print(f"После фильтрации: {len(df_filtered)}")
print()

group_a = df_filtered[df_filtered['release_month_clean'] == 12]['weighted_rating']
group_b = df_filtered[df_filtered['release_month_clean'] == 3]['weighted_rating']

print(f"Группа A (Декабрь): {len(group_a)} игр")
print(f"Группа B (Март): {len(group_b)} игр")

Всего PS4 игр: 694
После фильтрации: 229

Группа A (Декабрь): 19 игр
Группа B (Март): 20 игр


Игр маленькое количество, поэтому дальнейший анализ невозможен из-за маленькой выборки, поскольку маленькая выборка приводит к случайности результата.

**Вывод: для данной гипотезы невозможно провести A/B-тест.**


## Nintendo Switch: качество игр в декабре vs качество игр в сентябре

$H_0$: Качество игр в декабре и сентябре одинаковое.

$H_1$: Игры в декабре качественне, чем в сентябре.

In [155]:
df['is_switch'] = df['platforms'].str.contains('Nintendo Switch', case=False, na=False)
df_switch = df[df['is_switch'] == True].copy()


df_filtered = df_switch[
    (df_switch['ratings_count'] >= 10) &
    (df_switch['initial_price'] > 0) &
    (df_switch['platform_count'] <= 3)
]

print(f"Всего Switch игр: {len(df_switch)}")
print(f"После фильтрации: {len(df_filtered)}")
print()

group_a = df_filtered[df_filtered['release_month_clean'] == 7]['weighted_rating']
group_b = df_filtered[df_filtered['release_month_clean'] == 12]['weighted_rating']

print(f"Группа A (Июль): {len(group_a)} игр")
print(f"Группа B (Декабрь): {len(group_b)} игр")

Всего Switch игр: 608
После фильтрации: 69

Группа A (Июль): 3 игр
Группа B (Декабрь): 2 игр


Игр маленькое количество, поэтому дальнейший анализ невозможен из-за маленькой выборки, поскольку маленькая выборка приводит к случайности результата.

**Вывод: для данной гипотезы невозможно провести A/B-тест.**


## iOS: качество игр в декабре vs качество игр в июнь

$H_0$: Качество игр в декабре и июне одинаковое.

$H_1$: Игры в декабре качественне, чем в июне.

In [156]:
df['is_ios'] = df['platforms'].str.contains('IOS', case=False, na=False)
df_ios = df[df['is_ios'] == True].copy()

df_filtered = df_ios[
    (df_ios['ratings_count'] >= 10) &
    (df_ios['initial_price'] > 0) &
    (df_ios['platform_count'] <= 3)
]

print(f"Всего IOS игр: {len(df_ios)}")
print(f"После фильтрации: {len(df_filtered)}")
print()

group_a = df_filtered[df_filtered['release_month_clean'] == 6]['weighted_rating']
group_b = df_filtered[df_filtered['release_month_clean'] == 12]['weighted_rating']

print(f"Группа A (Июнь): {len(group_a)} игр")
print(f"Группа B (Декабрь): {len(group_b)} игр")

Всего IOS игр: 458
После фильтрации: 60

Группа A (Июнь): 2 игр
Группа B (Декабрь): 3 игр


Игр маленькое количество, поэтому дальнейший анализ невозможен из-за маленькой выборки, поскольку маленькая выборка приводит к случайности результата.

**Вывод: для данной гипотезы невозможно провести A/B-тест.**


# Выводы по результам тестирования

* Основная гипотеза про сезонность релизов подтвердилась: в апреле наблюдается наиболее большой спрос на релизы, в отличае от ноября.

* От конкуренции не зависит эффективность игры, то есть нет прямой зависимости между силой конкуренции и успешностью игры.

**Выводы для бизнеса:**

Напишите что-то пж пж пж